# Multiprocessing and Torchvision

PyTorch's torchvision preprocesses images using the PIL image procesing library. WebLoader can easily deal with this; you choose a decoder of type "PIL", the use the regular Torch preprocessing, and finally specify `converters="torch"`, which will return Torch tensors for all the elements.

Doing significant image processing during data augmentation is compute intensive, so you may want to use multiprocessing to speed this up; the `MultiWebLoader` class can perform this for you. This is similar to the use of multiprocessing in `torch.data`.

However, generally, it's a better choice (1) to do data augmentation offline prior to learning (if you have the storage) and (2) parallelize explicitly using the `tensorcom` package (rather than relying on Python `multiprocessing`).


In [1]:
!test -f training_ua.tgz || curl http://storage.googleapis.com/lpr-imagenet/imagenet_train-0000.tgz -o training_ua.tgz

In [2]:
source_url = "training_ua.tgz"
batch_size = 32

In [3]:
import webloader as wl
from torchvision import transforms

In [4]:
# standard torchvision-based ImageNet preprocessing

def info(x):
    print(type(x), x.shape)
    return x

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

preproc = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

In [5]:
training = wl.MultiWebLoader(
    source_url, 1000000,
    decode="PIL", # decode all image types to PIL
    fields="ppm;png;jpg cls", # extract these fields
    transforms=[preproc, None], # apply these transformations to the fields
    converters="torch", # shorthand for converters=[wl.totorch(), wl.totorch()]
    processes=4,
    batch_size=batch_size)

In [6]:
%%time
for i, b in enumerate(training):
    if i==0: print(b[0].shape, b[0].dtype)
    if i>=100: break

torch.Size([32, 3, 224, 224]) torch.float32
CPU times: user 102 ms, sys: 25 ms, total: 127 ms
Wall time: 10.7 s


In [7]:
training = wl.WebLoader(
    source_url, 1000000,
    decode="PIL", # decode all image types to PIL
    fields="ppm;png;jpg cls", # extract these fields
    transforms=[preproc, None], # apply these transformations to the fields
    converters="torch", # shorthand for converters=[wl.totorch(), wl.totorch()]
    batch_size=batch_size)

In [8]:
%%time
for i, b in enumerate(training):
    if i==0: print(b[0].shape, b[0].dtype, b[1].shape, b[1].dtype)
    if i>=100: break

torch.Size([32, 3, 224, 224]) torch.float32 torch.Size([32]) torch.int64
CPU times: user 1min 3s, sys: 437 ms, total: 1min 3s
Wall time: 34 s
